# Rapid prototype testing pipeline for Feldman lab
## ToDo

In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint

import spikeextractors as se
import spiketoolkit as st
import spikesorters as ss
import spikecomparison as sc
import spikewidgets as sw
import nwbwidgets

%matplotlib notebook

## 1a) Load short AP and LF recordings

In [ ]:
base_path = Path("D:/Feldman")
#base_path = Path("/Users/abuccino/Documents/Data/catalyst/brody/A256_bank1_2020_09_30_g0")
#base_data_path = Path("D:/Neuropixels/Neuropixels/A256_bank1_2020_09_30/A256_bank1_2020_09_30_g0")
base_data_path = Path("20210115_NPX_and_behavior/2021_01_15_E105/towersTask_g0")
ap_bin_path = base_data_path / "towersTask_g0_imec0" / "towersTask_g0_t0.imec0.ap.bin"
lf_bin_path = base_data_path / "towersTask_g0_imec0" / "towersTask_g0_t0.imec0.lf.bin"

### Make spikeinterface folders

In [ ]:
recording_folder = Path("20210115_NPX_and_behavior/2021_01_15_E105")
spikeinterface_folder = recording_folder / 'spikeinterface'base_path = Path("D:/Feldman")

spikeinterface_folder.mkdir(parents=True, exist_ok=True)

In [ ]:
recording_ap = se.SpikeGLXRecordingExtractor(ap_bin_path)
recording_lf = se.SpikeGLXRecordingExtractor(lf_bin_path)

### ToDo: perform quick spike evaluation?

In [ ]:
#quick_sort = ?

### save with behavioral data

In [ ]:
# The name of the NWBFile containing behavioral data
nwbfile_path = base_data_path / f"Feldman_prototype_{session_name}.nwb"

se.NwbSortingExtractor.write_recording(
    sorting=recording_ap,
    save_path=nwbfile_path,
    overwrite=False  # this appends the file. True would write a new file
)
se.NwbSortingExtractor.write_sorting(
    sorting=quick_sort,
    save_path=nwbfile_path,
    overwrite=False  # this appends the file. True would write a new file
)

# View output vs. behavior in NWBWidgets to ensure quality electrode placement
## ToDo